# OptionStatsViewer

This notebook browse the option statistics data that's simplified toa pickles file that's
pre-processed by the filter_historical.py / analyze_historical.py
It assume the pickles file contains a dataframe that has the historical optionstats of following fields
symbol, year, d_index(1 to 31), day_index(1 to 5), month, putiv, calliv, putoi, calloi, putvol, callvol, putoi, calloi
It then draw summarized plots that can be shown for each symbol

In [ ]:
import pandas as pd
import numpy as np
import os as os
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import json
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
historical_data_dir = "./data/historical/pickles"

In [ ]:
def read_data_file(filename):
    data_file = historical_data_dir + os.path.sep + filename
    if not os.path.exists(data_file):
        print("Error, historical data does not exist")

    df = pd.read_pickle(data_file)
    return df

### Some statistics

In [ ]:
def print_df_stats(df):
    print(df.shape)
    print(df.columns)
    print(df.index)
    print(np.unique(df.d_index))
    print(np.unique(df.day_index))
    print(np.unique(df.month))
    print(np.unique(df.year))

In [ ]:
df = read_data_file("2014_2018.pickle")
print_df_stats(df)

### Sample Aapl series

In [ ]:
aapl_series = df[df["symbol"]=='AAPL']

print(aapl_series.shape)
print(aapl_series.describe(include=np.number))

In [ ]:
fig = plt.figure(figsize=(40,20))

In [ ]:
def groupby_vol(df):
    df_symbol = df.groupby(by=['symbol','year', 'month']).agg(calliv_mean = ('calliv', np.mean),  putiv_mean=('putiv', np.mean), callvol_mean = ('callvol',np.mean),putvol_mean=('putvol', np.mean),calloi_mean=('calloi', np.mean),putoi_mean=('putoi', np.mean))
    return df_symbol

In [ ]:
df_symbol=groupby_vol(df)
df_symbol = df_symbol.reset_index()
df_symbol.to_csv("2014-2018_optionstats.csv", index=False)
print(df.columns)
print(df_symbol.index)
print(df_symbol.columns)
print(df_symbol.shape)

### Filter out symbols that has low volume (lower than average)

In [ ]:
def find_highvol_symbols(df_symbol):
    df_symbol["symbol"] = df_symbol.index
    callvol_mean_std = [df_symbol.callvol_mean.mean(), df_symbol.callvol_mean.std()]
    putvol_mean_std = [df_symbol.putvol_mean.mean(), df_symbol.putvol_mean.std()]
    calloi_mean_std = [df_symbol.calloi_mean.mean(), df_symbol.calloi_mean.std()]
    putoi_mean_std = [df_symbol.putoi_mean.mean(), df_symbol.putoi_mean.std()]
    tolerance_call = callvol_mean_std[0] - 0.05 * callvol_mean_std[1]
    tolerance_put = callvol_mean_std[0] - 0.05* putvol_mean_std[1]
    df_top_call_vol = df_symbol[df_symbol["callvol_mean"] >= tolerance_call]
    df_top_put_vol = df_symbol[df_symbol["putvol_mean"] >= tolerance_put]
    union_symbol_list =  list(set(df_top_call_vol.symbol) & set(df_top_put_vol.symbol))
    top_option_vol_df = df_symbol[df_symbol["symbol"].isin(union_symbol_list)]
    top_option_vol_df.index = range(0, top_option_vol_df.shape[0])
    return top_option_vol_df

In [ ]:
top_option_vol_df = find_highvol_symbols(df_symbol)
print(top_option_vol_df.shape)
top_option_vol_df.to_pickle(historical_data_dir + os.sep + "high_option_vol_2014_2018.pickle")

### Top Symbols for option volume
Now we have the top symbols that has highest option volumns (both call and put)

In [ ]:
pd.set_option("max_rows",100)
print(top_option_vol_df)

## High stock volume 

In [ ]:
def read_stock_stats(filename):
    stock_data_file = historical_data_dir + os.path.sep + filename
    if not os.path.exists(stock_data_file):
        print("Error, historical stock data does not exist")
    df_stock = pd.read_pickle(stock_data_file)
    return df_stock

In [ ]:
df_stock = read_stock_stats("2014_2018.pickle_stock")
print_df_stats(df_stock)

In [ ]:
aapl_series_stock = df_stock[df_stock["symbol"]=='AAPL']

print(aapl_series_stock.shape)
print(aapl_series_stock.describe(include=np.number))

In [ ]:
def aggr_stock_df(df_stock):
    df_symbol_stock = df_stock.groupby(by=['symbol']).agg(mean_close = ('close', np.mean), mean_vol =('volume', np.mean))
    df_symbol_stock["symbol"] = df_symbol_stock.index
    df_symbol_stock.index = range(0, df_symbol_stock.shape[0])
    return df_symbol_stock

In [ ]:
df_symbol_stock = aggr_stock_df(df_stock)
print(df_symbol_stock.head())

In [ ]:
def find_high_vol_stock(df_symbol_stock):
    vol_mean_std = [df_symbol_stock.mean_vol.mean(), df_symbol_stock.mean_vol.std()]
    df_top_stock = df_symbol_stock[df_symbol_stock["mean_vol"] >= vol_mean_std[0] - (0.05 * vol_mean_std[1])]
    df_top_stock.index = range(0, df_top_stock.index.size)
    return df_top_stock

In [ ]:
print(df_symbol_stock)
df_top_stock = find_high_vol_stock(df_symbol_stock)
#print(df_top_stock)

In [ ]:
pd.set_option("max_rows",df_top_stock.index.size)
print(df_top_stock)
print(df_top_stock.index.size)

In [ ]:
print(df_top_stock.symbol)

In [ ]:
df_top_stock.to_pickle("./data/high_stock_volume_2014_2018.pickle")

In [ ]:
def dump_json_file(res, f):
    with open(f, 'w') as (tFile):
        json.dump(res, tFile)

In [ ]:
high_vol_watch_list = list(set(df_top_stock.symbol) | set(top_option_vol_df))
dump_json_file(high_vol_watch_list, "./data/high_vol_watch_list.json")

In [ ]:
print(len(high_vol_watch_list))

### Harvest the detail level data

In [ ]:
df_2019 = read_data_file("2019_highvol.pickle")
print_df_stats(df_2019)

In [ ]:
df_symbol_2019 = groupby_vol(df_2019)
top_option_vol_df_2019 = find_highvol_symbols(df_symbol_2019)

In [ ]:
print(top_option_vol_df_2019.symbol)

In [ ]:
detail_stock_df = "highvol_2019.pickle_detail"

In [ ]:
detail_df = read_data_file(detail_stock_df)

In [ ]:
detail_df.describe()

In [ ]:
detail_df.columns

In [ ]:
def Filter_By_Strikes_Count(df, strikes_count):
    df = df.sort_values("Strike", ascending=True)
    unique_strikes = np.unique(df.Strike)
    median_strike = np.median(unique_strikes)
    index_of_median = np.floor(unique_strikes.size/2)
    strike_min = min(unique_strikes)
    strike_max = max(unique_strikes)
    if strikes_count < unique_strikes.size:
        strike_min = unique_strikes[np.int32(index_of_median - np.floor(strikes_count/2))]
        strike_max = unique_strikes[np.int32(index_of_median +  np.floor(strikes_count/2))] 
    print("strike min/max", strike_min, strike_max)
    return df[(df.Strike >= strike_min) & (df.Strike <= strike_max)]
    

In [ ]:
def filter_options_groupby(df, symbol, strikes_count):
    print(df.shape)
    selected_series =  df[df["UnderlyingSymbol"] == symbol]
    mean_vol = selected_series["Volume"].mean()
    selected_series = selected_series[selected_series["Volume"] >= mean_vol]
    selected_series = Filter_By_Strikes_Count(selected_series, 20)
    print("current shape after filter by strikes count", selected_series.shape)
    overall_vol_mean = selected_series.Volume.mean()
    overall_oi_mean = selected_series.OpenInterest.mean()
    print("Current series Vol mean, oi mean", overall_vol_mean, overall_oi_mean)
    df_groupby = selected_series.groupby(by=["OptionSymbol", "Type", "Strike", "Expiration"], as_index=True)
    pd.set_option("max_rows",selected_series.index.size)
    pd.set_option("max_columns",selected_series.columns.size)
    df_aggr = df_groupby.agg(vol_mean=('Volume', np.mean), oi_mean=('OpenInterest', np.mean), \
                  iv_mean=('IV', np.mean), delta_mean=('Delta', np.mean), theta_mean=('Theta', np.mean),\
                exp_day_max = ('days_to_expire', np.max), exp_day_min=('days_to_expire', np.min) ,\
                intrinsic_min=('intrinsic_value', np.min), intrinsic_max=('intrinsic_value', np.max) ,\
                time_min=('time_value', np.min), time_max=('time_value', np.max) )
    return df_aggr

In [ ]:
df_aggr = filter_options_groupby(detail_df, "AAPL", 20)

In [ ]:
print(df_aggr.columns)

In [ ]:
print(df_aggr.shape)
print(df_aggr.index)

In [ ]:
t =  df_aggr.reset_index()
print(t)

In [ ]:
interested_df = detail_df[detail_df["OptionSymbol"] == "AAPL200117C00190000"]

In [ ]:
print(interested_df.shape)
print(interested_df.columns)

In [ ]:
print(interested_df.sort_values("days_to_expire", ascending=False))

In [ ]:
print(detail_df[detail_df.OptionSymbol=='AAPL190118C00145000'].sort_values("days_to_expire", ascending=False))